# Table Merge

In [136]:
import os
import pandas as pd

PROCESSED_DATA_DIR = "../../data/processed data"
MERGED_DATA_DIR = "../../data/merged data"

SUB_FOLDER = "2020"

# Load Common Data

In [137]:
acs = pd.read_csv(os.path.join(PROCESSED_DATA_DIR, "census_tract_clusters.csv"))
# public_transportation = pd.read_csv(os.path.join(PROCESSED_DATA_DIR, "cleaned_public_transportation_data.csv"))
cta_L = pd.read_csv(os.path.join(PROCESSED_DATA_DIR, "cleaned_cta_L_data.csv"))
cta_bus = pd.read_csv(os.path.join(PROCESSED_DATA_DIR, "cleaned_cta_bus_data.csv"))

## TNP

In [138]:
tnp_trips = pd.read_csv(os.path.join(PROCESSED_DATA_DIR, SUB_FOLDER, "cleaned_tnp_data.csv"))
tnp_trips = tnp_trips[(tnp_trips["trip_start_time_of_day_f"] >= 6.0) & (tnp_trips["trip_start_time_of_day_f"] < 22.0)]
weather = pd.read_csv(os.path.join(PROCESSED_DATA_DIR, SUB_FOLDER, "cleaned_weather_data.csv"))
taxi_trips = pd.read_csv(os.path.join(PROCESSED_DATA_DIR, SUB_FOLDER, "cleaned_taxi_data.csv"))

taxi_trips = (
    taxi_trips
    .groupby(['trip_start_timestamp', 'pickup_census_tract', 'dropoff_census_tract'])
    .size()
    .reset_index(name='taxi')
)

tnp_trips["pickup_census_tract"] = tnp_trips["pickup_census_tract"].astype(int)
acs["GEOID"] = acs["GEOID"].astype(int)

# tnp_trips['trip_start_date'] = pd.to_datetime(tnp_trips['trip_start_date']).dt.date
# cta_L['date'] = pd.to_datetime(cta_L['date']).dt.date
# cta_bus['date'] = pd.to_datetime(cta_bus['date']).dt.date

merged_tnp_data = tnp_trips.merge(acs, left_on="pickup_census_tract", right_on="GEOID", how="left")
merged_tnp_data = merged_tnp_data.merge(weather, left_on="trip_start_date", right_on="Date", how="left")
# merged_tnp_data = merged_tnp_data.merge(public_transportation, left_on="trip_start_date", right_on="service_date", how="left")
merged_tnp_data = merged_tnp_data.merge(cta_L, left_on=["pickup_census_tract", "trip_start_date"], right_on=["GEOID10", "date"], how="left")
merged_tnp_data = merged_tnp_data.merge(cta_bus, left_on=["pickup_census_tract", "trip_start_date"], right_on=["GEOID10", "date"], how="left")
merged_tnp_data = merged_tnp_data.merge(
    taxi_trips,
    left_on=['trip_start_timestamp', 'pickup_census_tract', 'dropoff_census_tract'],
    right_on=['trip_start_timestamp', 'pickup_census_tract', 'dropoff_census_tract'],
    how="left"
)

### Data Cleaning

In [139]:
VARIABLES = {
    "MEDINC": "DP03_0062E",          # Median Household Income
    "INCPERCAP": "DP03_0088E",       # Per Capita Income
    "INC_LT10K": "DP03_0052E",       # Income < $10k
    "INC_10_15K": "DP03_0053E",
    "INC_15_25K": "DP03_0054E",
    "INC_25_35K": "DP03_0055E",
    "INC_35_50K": "DP03_0056E",
    "INC_50_75K": "DP03_0057E",
    "INC_75_100K": "DP03_0058E",
    "INC_100_150K": "DP03_0059E",
    "INC_150_200K": "DP03_0060E",
    "INC_GT200K": "DP03_0061E",
    "POVERTY_RATE": "DP03_0119PE",   # % Below Poverty Line
    "SNAP": "DP03_0074E",            # Households with Food Stamps/SNAP
    "SS_INCOME": "DP03_0066E",       # Households with Social Security Income
}

merged_tnp_data = merged_tnp_data.dropna(subset=list(VARIABLES.keys()))

merged_tnp_data["day_type"] = merged_tnp_data["daytype"]
merged_tnp_data = merged_tnp_data[merged_tnp_data["day_type"] == 'W']

merged_tnp_data['taxi'] = merged_tnp_data['taxi'].fillna(0).astype(int)

### Drop Duplicate Feature

In [140]:
merged_tnp_data = merged_tnp_data.drop(columns=[
                                        "trip_id", "trip_end_timestamp",
                                        "pickup_centroid_latitude", "pickup_centroid_longitude",
                                        "dropoff_centroid_latitude", "dropoff_centroid_longitude",
                                        "trip_end_date",
                                        "nonworkday",
                                        "GEOID",
                                        "day_type",
                                        "shared_trip_authorized", "trips_pooled",
                                        "pickup_centroid_location", "dropoff_centroid_location",
                                        "Date", "Station",
                                        # "service_date",
                                        "WSI",
                                        "GEOID10_x", "date_x", "GEOID10_y", "date_y",
                                        ])

merged_tnp_data['total_rides'] = merged_tnp_data['rides'] + merged_tnp_data['daily_bus_rides']

## Manipulation

In [141]:
def classify_area(pickup, dropoff):
    loop_area = 32
    special_areas = [76, 8, 56, 33]  # O'Hare, Near North Side, Garfield Ridge, Near South Side

    if loop_area in [pickup, dropoff]:
        return 1  # Loop
    elif pickup in special_areas or dropoff in special_areas:
        return 2  # Other target zones
    else:
        return 0  # Other

# 确保区域编号为整数
merged_tnp_data["pickup_community_area"] = merged_tnp_data["pickup_community_area"].astype(int)
merged_tnp_data["dropoff_community_area"] = merged_tnp_data["dropoff_community_area"].astype(int)

# 应用分类函数
merged_tnp_data["area_type"] = merged_tnp_data.apply(
    lambda row: classify_area(row["pickup_community_area"], row["dropoff_community_area"]),
    axis=1
)

# 添加 day_of_week（如原始代码）
merged_tnp_data["trip_start_timestamp"] = pd.to_datetime(merged_tnp_data["trip_start_timestamp"])
merged_tnp_data["day_of_week"] = merged_tnp_data["trip_start_timestamp"].dt.dayofweek

In [142]:
# 检查是否存在 NaN
has_nan = merged_tnp_data["total_rides"].isna().any()
nan_count = merged_tnp_data["total_rides"].isna().sum()

print(f"是否存在 NaN: {has_nan}")
print(f"NaN 总数: {nan_count}")

# 查看最大值、最小值、均值等
print("\n total_rides 统计信息：")
print(merged_tnp_data["total_rides"].describe())

是否存在 NaN: False
NaN 总数: 0

 total_rides 统计信息：
count    1.705779e+06
mean     5.263564e+04
std      7.371695e+04
min      6.163831e+02
25%      8.951843e+03
50%      1.489467e+04
75%      4.943864e+04
max      2.202542e+05
Name: total_rides, dtype: float64


### Save Data

In [143]:
merged_tnp_data.to_csv(os.path.join(MERGED_DATA_DIR, SUB_FOLDER, "merged_tnp_data.csv"), index=False)